In [311]:
import pandas as pd
import numpy as np

In [312]:
# importando os dados de docentes
dados2019 = pd.read_csv("docente2019.csv", sep="|", encoding="latin 1")

In [313]:
# Importando tabela com dados das IES
dadosies2019 = pd.read_csv("SUP_IES_2019.csv", sep="|", encoding="latin 1")

In [433]:
# Importando tabela com dados de raça da população por UF
esperado_raca_uf = pd.read_csv("esperado_raca_uf.csv")

In [434]:
esperado_raca_uf.rename({"UF": "nome_uf"}, axis=1, inplace=True)

In [368]:
pd.options.display.float_format = '{:,.4f}'.format

In [314]:
# visualizando os dados
dados2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399428 entries, 0 to 399427
Data columns (total 41 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   NU_ANO_CENSO                  399428 non-null  int64  
 1   CO_IES                        399428 non-null  int64  
 2   TP_CATEGORIA_ADMINISTRATIVA   399428 non-null  int64  
 3   TP_ORGANIZACAO_ACADEMICA      399428 non-null  int64  
 4   CO_DOCENTE_IES                399428 non-null  int64  
 5   ID_DOCENTE                    399428 non-null  object 
 6   TP_SITUACAO                   399428 non-null  int64  
 7   TP_ESCOLARIDADE               399428 non-null  int64  
 8   TP_REGIME_TRABALHO            386073 non-null  float64
 9   TP_SEXO                       399428 non-null  int64  
 10  NU_ANO_NASCIMENTO             399428 non-null  int64  
 11  NU_MES_NASCIMENTO             399428 non-null  int64  
 12  NU_DIA_NASCIMENTO             399428 non-nul

In [315]:
dadosies2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2608 entries, 0 to 2607
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   NU_ANO_CENSO                    2608 non-null   int64  
 1   CO_IES                          2608 non-null   int64  
 2   NO_IES                          2608 non-null   object 
 3   SG_IES                          2144 non-null   object 
 4   CO_MANTENEDORA                  2608 non-null   int64  
 5   NO_MANTENEDORA                  2608 non-null   object 
 6   TP_CATEGORIA_ADMINISTRATIVA     2608 non-null   int64  
 7   TP_ORGANIZACAO_ACADEMICA        2608 non-null   int64  
 8   CO_REGIAO                       2608 non-null   int64  
 9   CO_UF                           2608 non-null   int64  
 10  CO_MUNICIPIO                    2608 non-null   int64  
 11  IN_CAPITAL                      2608 non-null   int64  
 12  QT_TEC_TOTAL                    26

## Ajustes que vão facilitar as análises:
### Incluir colunas com a descrição da raça e da categoria administrativa, além do código e do nome da UF 

In [572]:
# Padronizando: transformar para string o que não é número
dados2019["CO_IES"] = dados2019["CO_IES"].astype(str)
dados2019["TP_CATEGORIA_ADMINISTRATIVA"] = dados2019["TP_CATEGORIA_ADMINISTRATIVA"].astype(str)
dadosies2019["TP_CATEGORIA_ADMINISTRATIVA"] = dadosies2019["TP_CATEGORIA_ADMINISTRATIVA"].astype(str)
dadosies2019["CO_UF"] = dadosies2019["CO_UF"].astype(str)
dadosies2019["CO_IES"] = dadosies2019["CO_IES"].astype(str)

In [317]:
# Criando a coluna descricao_raca para facilitar a analise
dados2019['DESC_RACA'] = dados2019['TP_COR_RACA'].replace([0, 1, 2, 3, 4, 5, 9],['nao consta', 'branca', 'preta', 'parda', 'amarela', 'indigena', 'nao consta'])

In [320]:
# selecionando o que vou usar
ies_e_uf = dadosies2019.loc[:,["CO_IES", "CO_UF", "SG_IES", "NO_IES"]]

In [323]:
# juntando porque nao tem uf na outra tabela
dados2019 = pd.merge(dados2019, ies_e_uf, on=["CO_IES"], how="left")

In [573]:
# Criando uma nova coluna com o nome da categoria administrativa
dados2019['NOME_CATEGORIA_ADMINISTRATIVA'] = dados2019['TP_CATEGORIA_ADMINISTRATIVA'].replace(["1", "2", "3", "4", "5", "6", "7", "8", "9"],["federal", "estadual", "municipal", "privada com fins lucrativos", "privada sem fins lucrativos", "particular estrita", "especial", "privada comunitária", "privada confessional"])
dadosies2019['NOME_CATEGORIA_ADMINISTRATIVA'] = dadosies2019['TP_CATEGORIA_ADMINISTRATIVA'].replace(["1", "2", "3", "4", "5", "6", "7", "8", "9"],["federal", "estadual", "municipal", "privada com fins lucrativos", "privada sem fins lucrativos", "particular estrita", "especial", "privada comunitária", "privada confessional"])

In [324]:
# traduzindo as ufs e organizando a tabela
uf_e_codigo = pd.read_csv("ufs_e_codigos.csv")
uf_e_codigo.rename({"Códigos": "CO_UF", "UFs": "nome_uf"}, axis=1, inplace=True)
uf_e_codigo["CO_UF"] = uf_e_codigo["CO_UF"].astype(str)
dados2019 = pd.merge(dados2019, uf_e_codigo, on=["CO_UF"], how="left")

# Por IES
### Agora vamos fazer uma tabela dinâmica para ver a distribuição por raça em cada IES. Vamos ver o número bruto e o porcentual.

In [325]:
dados2019_raca = dados2019.copy()

In [326]:
# tabela dinâmica
dados2019_raca = dados2019_raca.pivot_table(index="CO_IES",
                           columns="DESC_RACA",
                           values="ID_DOCENTE",
                           aggfunc="count")

In [327]:
# ajustando para ficar legível para a máquina
dados2019_raca = pd.DataFrame(dados2019_raca.to_records())

In [328]:
# somando o número de professores
dados2019_raca["TOTAL_PROF"] = dados2019_raca.sum(axis=1)

/var/folders/vc/cyjw97ds7_s2zld8ppcly6340000gn/T/ipykernel_8378/1704349015.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dados2019_raca["TOTAL_PROF"] = dados2019_raca.sum(axis=1)


In [330]:
# trocando os valores números por 0 pra não dar erro nas somas
dados2019_raca.fillna(0, inplace=True)

In [331]:
dados2019_raca

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF
0,1,148.0000,"1,380.0000",6.0000,246.0000,576.0000,4.0000,"2,360.0000"
1,10,31.0000,807.0000,1.0000,514.0000,45.0000,14.0000,"1,412.0000"
2,10016,0.0000,3.0000,0.0000,0.0000,3.0000,0.0000,6.0000
3,10058,0.0000,7.0000,0.0000,15.0000,16.0000,4.0000,42.0000
4,10071,0.0000,22.0000,0.0000,0.0000,1.0000,0.0000,23.0000
...,...,...,...,...,...,...,...,...
2603,989,0.0000,11.0000,0.0000,1.0000,3.0000,0.0000,15.0000
2604,99,0.0000,22.0000,0.0000,17.0000,6.0000,1.0000,46.0000
2605,991,0.0000,31.0000,0.0000,6.0000,8.0000,2.0000,47.0000
2606,994,0.0000,8.0000,0.0000,12.0000,5.0000,1.0000,26.0000


In [332]:
# novo dataframe chamado "dados2019_raca_porc". assim ficamos com: um df com o valor bruto e um com o porcentual
dados2019_raca_porc = dados2019_raca.copy()

In [333]:
dados2019_raca_porc

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF
0,1,148.0000,"1,380.0000",6.0000,246.0000,576.0000,4.0000,"2,360.0000"
1,10,31.0000,807.0000,1.0000,514.0000,45.0000,14.0000,"1,412.0000"
2,10016,0.0000,3.0000,0.0000,0.0000,3.0000,0.0000,6.0000
3,10058,0.0000,7.0000,0.0000,15.0000,16.0000,4.0000,42.0000
4,10071,0.0000,22.0000,0.0000,0.0000,1.0000,0.0000,23.0000
...,...,...,...,...,...,...,...,...
2603,989,0.0000,11.0000,0.0000,1.0000,3.0000,0.0000,15.0000
2604,99,0.0000,22.0000,0.0000,17.0000,6.0000,1.0000,46.0000
2605,991,0.0000,31.0000,0.0000,6.0000,8.0000,2.0000,47.0000
2606,994,0.0000,8.0000,0.0000,12.0000,5.0000,1.0000,26.0000


In [334]:
# ajustando as colunas porcentuais
dados2019_raca_porc["negra"] = (dados2019_raca_porc["preta"] + dados2019_raca_porc["parda"])/dados2019_raca_porc["TOTAL_PROF"]
dados2019_raca_porc["amarela"] = dados2019_raca_porc["amarela"] / dados2019_raca_porc["TOTAL_PROF"]
dados2019_raca_porc["branca"] = dados2019_raca_porc["branca"] / dados2019_raca_porc["TOTAL_PROF"]
dados2019_raca_porc["indigena"] = dados2019_raca_porc["indigena"] / dados2019_raca_porc["TOTAL_PROF"]
dados2019_raca_porc["nao consta"] = dados2019_raca_porc["nao consta"] / dados2019_raca_porc["TOTAL_PROF"]
dados2019_raca_porc["parda"] = dados2019_raca_porc["parda"] / dados2019_raca_porc["TOTAL_PROF"]
dados2019_raca_porc["preta"] = dados2019_raca_porc["preta"] / dados2019_raca_porc["TOTAL_PROF"]

In [335]:
dados2019_raca_porc

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra
0,1,0.0627,0.5847,0.0025,0.1042,0.2441,0.0017,"2,360.0000",0.2458
1,10,0.0220,0.5715,0.0007,0.3640,0.0319,0.0099,"1,412.0000",0.0418
2,10016,0.0000,0.5000,0.0000,0.0000,0.5000,0.0000,6.0000,0.5000
3,10058,0.0000,0.1667,0.0000,0.3571,0.3810,0.0952,42.0000,0.4762
4,10071,0.0000,0.9565,0.0000,0.0000,0.0435,0.0000,23.0000,0.0435
...,...,...,...,...,...,...,...,...,...
2603,989,0.0000,0.7333,0.0000,0.0667,0.2000,0.0000,15.0000,0.2000
2604,99,0.0000,0.4783,0.0000,0.3696,0.1304,0.0217,46.0000,0.1522
2605,991,0.0000,0.6596,0.0000,0.1277,0.1702,0.0426,47.0000,0.2128
2606,994,0.0000,0.3077,0.0000,0.4615,0.1923,0.0385,26.0000,0.2308


In [336]:
# vendo as estatísticas
dados2019_raca_porc.describe().T

,count,mean,std,min,25%,50%,75%,max
amarela,"2,608.0000",0.0083,0.0221,0.0000,0.0000,0.0000,0.0073,0.4000
branca,"2,608.0000",0.5782,0.2695,0.0000,0.3589,0.6035,0.8177,1.0000
indigena,"2,608.0000",0.0010,0.0071,0.0000,0.0000,0.0000,0.0000,0.2000
nao consta,"2,608.0000",0.1923,0.2207,0.0000,0.0248,0.1113,0.2833,1.0000
parda,"2,608.0000",0.1934,0.1992,0.0000,0.0336,0.1250,0.3077,1.0000
preta,"2,608.0000",0.0268,0.0535,0.0000,0.0000,0.0071,0.0345,1.0000
TOTAL_PROF,"2,608.0000",153.1549,389.8369,1.0000,18.0000,43.0000,113.0000,"5,924.0000"
negra,"2,608.0000",0.2201,0.2125,0.0000,0.0476,0.1495,0.3420,1.0000


## Por categoria administrativa

### Quero ver as estatísticas de raça por categoria administrativa:
#### - Excluir repetidos;
#### - Agrupar por categoria administrativa

In [42]:
# Criando o dataframe

dados2019_catadmin = dados2019.copy()

Há professores que trabalham em mais de uma universidade de diferentes categorias administrativas. Por isso, excluir os duplicados pelo critério "ID_DOCENTE" não se aplica nesta análise. Vou criar uma nova coluna juntando o ID_DOCENTE com a categoria administrativa e aí sim excluir por esta coluna. Dessa forma, se um docente der aula em duas universidades privadas, será computado apenas uma vez. Mas se ele der aula em uma pública e uma privada será computado nas duas categorias.

In [341]:
#Excluir repetidos 1: Juntando categoria administrativa com id_docente

dados2019_catadmin["ID_E_CATEGORIA"] = dados2019_catadmin["TP_CATEGORIA_ADMINISTRATIVA"] + dados2019_catadmin["ID_DOCENTE"]

In [344]:
# Excluindo repetidos
dados2019_catadmin.drop_duplicates("ID_E_CATEGORIA", keep="last", inplace=True)

In [345]:
# Agrupando

pvt_table_catadmin = dados2019_catadmin.pivot_table(index="NOME_CATEGORIA_ADMINISTRATIVA",
                             columns="DESC_RACA",
                             values="ID_E_CATEGORIA",
                             aggfunc="count")

In [346]:
pvt_table_catadmin = pd.DataFrame(pvt_table_catadmin.to_records())

In [347]:
pvt_table_catadmin["total_prof"] = pvt_table_catadmin.sum(axis=1)

/var/folders/vc/cyjw97ds7_s2zld8ppcly6340000gn/T/ipykernel_8378/3730363661.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pvt_table_catadmin["total_prof"] = pvt_table_catadmin.sum(axis=1)


In [348]:
pvt_table_catadmin_porc = pvt_table_catadmin.copy()

In [350]:
# ajustando as colunas porcentuais
pvt_table_catadmin_porc["amarela"] = pvt_table_catadmin_porc["amarela"] / pvt_table_catadmin_porc["total_prof"]
pvt_table_catadmin_porc["branca"] = pvt_table_catadmin_porc["branca"] / pvt_table_catadmin_porc["total_prof"]
pvt_table_catadmin_porc["indigena"] = pvt_table_catadmin_porc["indigena"] / pvt_table_catadmin_porc["total_prof"]
pvt_table_catadmin_porc["nao consta"] = pvt_table_catadmin_porc["nao consta"] / pvt_table_catadmin_porc["total_prof"]
pvt_table_catadmin_porc["parda"] = pvt_table_catadmin_porc["parda"] / pvt_table_catadmin_porc["total_prof"]
pvt_table_catadmin_porc["preta"] = pvt_table_catadmin_porc["preta"] / pvt_table_catadmin_porc["total_prof"]
pvt_table_catadmin_porc["negra"] = pvt_table_catadmin_porc["parda"] + pvt_table_catadmin_porc["preta"]

In [351]:
pvt_table_catadmin_porc

,NOME_CATEGORIA_ADMINISTRATIVA,amarela,branca,indigena,nao consta,parda,preta,total_prof,negra
0,especial,0.0060,0.6284,0.0022,0.1301,0.2108,0.0225,1822,0.2333
1,estadual,0.0130,0.4724,0.0015,0.3673,0.1245,0.0211,50832,0.1457
2,federal,0.0104,0.3907,0.0018,0.4395,0.1339,0.0237,126382,0.1576
3,municipal,0.0210,0.8015,0.0002,0.1190,0.0480,0.0104,4815,0.0584
4,privada com fins lucrativos,0.0089,0.5418,0.0008,0.2232,0.2055,0.0199,91253,0.2253
5,privada sem fins lucrativos,0.0087,0.6713,0.0010,0.1891,0.1151,0.0147,96229,0.1298


In [352]:
pvt_table_catadmin

,NOME_CATEGORIA_ADMINISTRATIVA,amarela,branca,indigena,nao consta,parda,preta,total_prof
0,especial,11,1145,4,237,384,41,1822
1,estadual,662,24014,78,18672,6331,1075,50832
2,federal,1311,49373,231,55545,16925,2997,126382
3,municipal,101,3859,1,573,231,50,4815
4,privada com fins lucrativos,816,49437,69,20370,18749,1812,91253
5,privada sem fins lucrativos,841,64603,101,18193,11073,1418,96229


In [353]:
pvt_table_catadmin_porc.describe().T

,count,mean,std,min,25%,50%,75%,max
amarela,6.0000,0.0113,0.0052,0.0060,0.0088,0.0097,0.0124,0.0210
branca,6.0000,0.5843,0.1473,0.3907,0.4898,0.5851,0.6606,0.8015
indigena,6.0000,0.0013,0.0007,0.0002,0.0008,0.0013,0.0018,0.0022
nao consta,6.0000,0.2447,0.1308,0.1190,0.1448,0.2061,0.3313,0.4395
parda,6.0000,0.1396,0.0611,0.0480,0.1174,0.1292,0.1876,0.2108
preta,6.0000,0.0187,0.0051,0.0104,0.0160,0.0205,0.0222,0.0237
total_prof,6.0000,"61,888.8333","51,358.6806","1,822.0000","16,319.2500","71,042.5000","94,985.0000","126,382.0000"
negra,6.0000,0.1583,0.0649,0.0584,0.1338,0.1517,0.2084,0.2333


## Por UF

#### - Excluir repetidos;
#### - Agrupar por UF

In [354]:
# criando o df
dados2019_uf = dados2019.copy()

In [355]:
# Juntando UF com id_docente para excluir repetidos
dados2019_uf["ID_E_UF"] = dados2019_uf["CO_UF"] + dados2019_uf["ID_DOCENTE"]

In [356]:
# Excluindo repetidos
dados2019_uf.drop_duplicates("ID_E_UF", keep="last", inplace=True)

In [357]:
# Agrupando
pvt_table_uf = dados2019_uf.pivot_table(index="nome_uf",
                             columns="DESC_RACA",
                             values="ID_E_UF",
                             aggfunc="count")

In [358]:
# ajustando para ficar legível para a máquina
pvt_table_uf = pd.DataFrame(pvt_table_uf.to_records())

In [359]:
pvt_table_uf["TOTAL_PROF"] = pvt_table_uf.sum(axis=1)

/var/folders/vc/cyjw97ds7_s2zld8ppcly6340000gn/T/ipykernel_8378/2138595117.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pvt_table_uf["TOTAL_PROF"] = pvt_table_uf.sum(axis=1)


In [362]:
pvt_table_uf_porc = pvt_table_uf.copy()

In [363]:
# ajustando as colunas porcentuais
pvt_table_uf_porc["amarela"] = pvt_table_uf_porc["amarela"] / pvt_table_uf_porc["TOTAL_PROF"]
pvt_table_uf_porc["branca"] = pvt_table_uf_porc["branca"] / pvt_table_uf_porc["TOTAL_PROF"]
pvt_table_uf_porc["indigena"] = pvt_table_uf_porc["indigena"] / pvt_table_uf_porc["TOTAL_PROF"]
pvt_table_uf_porc["nao consta"] = pvt_table_uf_porc["nao consta"] / pvt_table_uf_porc["TOTAL_PROF"]
pvt_table_uf_porc["parda"] = pvt_table_uf_porc["parda"] / pvt_table_uf_porc["TOTAL_PROF"]
pvt_table_uf_porc["preta"] = pvt_table_uf_porc["preta"] / pvt_table_uf_porc["TOTAL_PROF"]
pvt_table_uf_porc["negra"] = pvt_table_uf_porc["parda"] + pvt_table_uf_porc["preta"]

In [364]:
pvt_table_uf_porc

,nome_uf,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra
0,Acre,0.0046,0.2344,0.0007,0.3519,0.3841,0.0243,1523,0.4084
1,Alagoas,0.0116,0.4270,0.0013,0.1008,0.4272,0.0321,4731,0.4593
2,Amapá,0.0080,0.3551,0.0046,0.1123,0.4593,0.0607,1746,0.5200
3,Amazonas,0.0042,0.1702,0.0026,0.5598,0.2505,0.0127,5030,0.2632
4,Bahia,0.0084,0.2757,0.0021,0.3909,0.2828,0.0400,18619,0.3228
5,Ceará,0.0100,0.3226,0.0027,0.3339,0.3099,0.0210,12750,0.3309
6,Distrito Federal,0.0113,0.5021,0.0013,0.1957,0.2646,0.0249,8419,0.2896
7,Espírito Santo,0.0086,0.6372,0.0021,0.1340,0.1906,0.0276,6711,0.2181
8,Goiás,0.0076,0.3613,0.0006,0.4570,0.1506,0.0229,12638,0.1735
9,Maranhão,0.0087,0.3442,0.0019,0.2075,0.3761,0.0616,6993,0.4377


In [365]:
pvt_table_uf

,nome_uf,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF
0,Acre,7,357,1,536,585,37,1523
1,Alagoas,55,2020,6,477,2021,152,4731
2,Amapá,14,620,8,196,802,106,1746
3,Amazonas,21,856,13,2816,1260,64,5030
4,Bahia,157,5134,40,7278,5265,745,18619
5,Ceará,127,4113,34,4257,3951,268,12750
6,Distrito Federal,95,4227,11,1648,2228,210,8419
7,Espírito Santo,58,4276,14,899,1279,185,6711
8,Goiás,96,4566,8,5775,1903,290,12638
9,Maranhão,61,2407,13,1451,2630,431,6993


## Geral e por gênero

In [366]:
dados2019_unicos = dados2019.copy()

In [367]:
dados2019_unicos.drop_duplicates("ID_DOCENTE", keep="last", inplace=True)

In [369]:
# geral porcentagem
dados2019_unicos["DESC_RACA"].value_counts(normalize=True)

branca       0.5181
nao consta   0.3081
parda        0.1424
preta        0.0199
amarela      0.0101
indigena     0.0013
Name: DESC_RACA, dtype: float64

In [370]:
negra = 0.1424 + 0.0199
negra

0.1623

In [371]:
# geral valor bruto
dados2019_unicos["DESC_RACA"].value_counts()

branca        182149
nao consta    108329
parda          50074
preta           7004
amarela         3565
indigena         462
Name: DESC_RACA, dtype: int64

In [372]:
# trocando as variáveis da categoria TP Sexo para facilitar a análise
dados2019_unicos["TP_SEXO"] = dados2019_unicos["TP_SEXO"].replace([1, 2], ["F", "M"])

In [373]:
# Agrupando por gênero

pvt_table_genero = dados2019_unicos.pivot_table(index="TP_SEXO",
                             columns="DESC_RACA",
                             values="ID_DOCENTE",
                             aggfunc="count")

In [374]:
pvt_table_genero = pd.DataFrame((pvt_table_genero).to_records())

In [375]:
pvt_table_genero["total_sexo"] = pvt_table_genero.sum(axis=1)

/var/folders/vc/cyjw97ds7_s2zld8ppcly6340000gn/T/ipykernel_8378/511801599.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pvt_table_genero["total_sexo"] = pvt_table_genero.sum(axis=1)


In [376]:
pvt_table_genero_porc = pvt_table_genero.copy()

In [377]:
# ajustando as colunas porcentuais
pvt_table_genero_porc["amarela"] = pvt_table_genero_porc["amarela"] / pvt_table_genero_porc["total_sexo"]
pvt_table_genero_porc["branca"] = pvt_table_genero_porc["branca"] / pvt_table_genero_porc["total_sexo"]
pvt_table_genero_porc["indigena"] = pvt_table_genero_porc["indigena"] / pvt_table_genero_porc["total_sexo"]
pvt_table_genero_porc["nao consta"] = pvt_table_genero_porc["nao consta"] / pvt_table_genero_porc["total_sexo"]
pvt_table_genero_porc["parda"] = pvt_table_genero_porc["parda"] / pvt_table_genero_porc["total_sexo"]
pvt_table_genero_porc["preta"] = pvt_table_genero_porc["preta"] / pvt_table_genero_porc["total_sexo"]
pvt_table_genero_porc["negra"] = pvt_table_genero_porc["parda"] + pvt_table_genero_porc["preta"]

In [378]:
pvt_table_genero_porc

,TP_SEXO,amarela,branca,indigena,nao consta,parda,preta,total_sexo,negra
0,F,0.0099,0.5206,0.0011,0.3089,0.1412,0.0183,164755,0.1595
1,M,0.0103,0.5159,0.0015,0.3074,0.1435,0.0213,186828,0.1649


In [379]:
pvt_table_genero

,TP_SEXO,amarela,branca,indigena,nao consta,parda,preta,total_sexo
0,F,1633,85771,185,50889,23256,3021,164755
1,M,1932,96378,277,57440,26818,3983,186828


## Escolaridade

In [406]:
dados_escolaridade = dados2019_unicos.copy()

In [407]:
dados_escolaridade["TP_ESCOLARIDADE"] = dados_escolaridade["TP_ESCOLARIDADE"].replace([1, 2, 3, 4, 5], ["sem graduacao", "graduacao", "especializacao", "mestrado", "doutorado"])

In [408]:
pvt_table_escolaridade = dados_escolaridade.pivot_table(index="TP_ESCOLARIDADE",
                             columns="DESC_RACA",
                             values="ID_DOCENTE",
                             aggfunc="count")

In [409]:
pvt_table_escolaridade = pd.DataFrame(pvt_table_escolaridade.to_records())

In [410]:
pvt_table_escolaridade

,TP_ESCOLARIDADE,amarela,branca,indigena,nao consta,parda,preta
0,doutorado,"1,842.0000","79,943.0000",243.0000,"59,100.0000","16,747.0000","2,757.0000"
1,especializacao,526.0000,"30,983.0000",49.0000,"14,353.0000","10,853.0000","1,292.0000"
2,graduacao,52.0000,"1,160.0000",3.0000,"1,521.0000",686.0000,113.0000
3,mestrado,"1,145.0000","70,059.0000",167.0000,"33,351.0000","21,785.0000","2,841.0000"
4,sem graduacao,NaN,4.0000,NaN,4.0000,3.0000,1.0000


In [411]:
pvt_table_escolaridade["TP_ESCOLARIDADE"] = pvt_table_escolaridade["TP_ESCOLARIDADE"].astype(str)

In [412]:
pvt_table_escolaridade["TOTAL_PROF"] = pvt_table_escolaridade.sum(axis=1)

/var/folders/vc/cyjw97ds7_s2zld8ppcly6340000gn/T/ipykernel_8378/3823658858.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pvt_table_escolaridade["TOTAL_PROF"] = pvt_table_escolaridade.sum(axis=1)


In [413]:
pvt_table_escolaridade_porc = pvt_table_escolaridade.copy()

In [414]:
# ajustando as colunas porcentuais
pvt_table_escolaridade_porc["negra"] = (pvt_table_escolaridade_porc["preta"] + pvt_table_escolaridade_porc["parda"]) / 57078
pvt_table_escolaridade_porc["amarela"] = pvt_table_escolaridade_porc["amarela"] / 3565
pvt_table_escolaridade_porc["branca"] = pvt_table_escolaridade_porc["branca"] / 182149
pvt_table_escolaridade_porc["indigena"] = pvt_table_escolaridade_porc["indigena"] / 462
pvt_table_escolaridade_porc["nao consta"] = pvt_table_escolaridade_porc["nao consta"] / pvt_table_escolaridade_porc["TOTAL_PROF"]
pvt_table_escolaridade_porc["parda"] = pvt_table_escolaridade_porc["parda"] / 50074
pvt_table_escolaridade_porc["preta"] = pvt_table_escolaridade_porc["preta"] / 7004


In [415]:
pvt_table_escolaridade_porc

,TP_ESCOLARIDADE,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra
0,doutorado,0.5167,0.4389,0.5260,0.3679,0.3344,0.3936,"160,632.0000",0.3417
1,especializacao,0.1475,0.1701,0.1061,0.2472,0.2167,0.1845,"58,056.0000",0.2128
2,graduacao,0.0146,0.0064,0.0065,0.4303,0.0137,0.0161,"3,535.0000",0.0140
3,mestrado,0.3212,0.3846,0.3615,0.2578,0.4351,0.4056,"129,348.0000",0.4314
4,sem graduacao,NaN,0.0000,NaN,0.3333,0.0001,0.0001,12.0000,0.0001


In [416]:
pvt_table_escolaridade

,TP_ESCOLARIDADE,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF
0,doutorado,"1,842.0000","79,943.0000",243.0000,"59,100.0000","16,747.0000","2,757.0000","160,632.0000"
1,especializacao,526.0000,"30,983.0000",49.0000,"14,353.0000","10,853.0000","1,292.0000","58,056.0000"
2,graduacao,52.0000,"1,160.0000",3.0000,"1,521.0000",686.0000,113.0000,"3,535.0000"
3,mestrado,"1,145.0000","70,059.0000",167.0000,"33,351.0000","21,785.0000","2,841.0000","129,348.0000"
4,sem graduacao,NaN,4.0000,NaN,4.0000,3.0000,1.0000,12.0000


## Comparação entre a porcentagem de professores negros da universidade com a porcentagem de habitantes negros na UF

In [574]:
# criando nova tabela a partir da tabela de raça por ies
dados2019_difporc = dados2019_raca_porc.copy()

In [575]:
dados2019_difporc

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra
0,1,0.0627,0.5847,0.0025,0.1042,0.2441,0.0017,"2,360.0000",0.2458
1,10,0.0220,0.5715,0.0007,0.3640,0.0319,0.0099,"1,412.0000",0.0418
2,10016,0.0000,0.5000,0.0000,0.0000,0.5000,0.0000,6.0000,0.5000
3,10058,0.0000,0.1667,0.0000,0.3571,0.3810,0.0952,42.0000,0.4762
4,10071,0.0000,0.9565,0.0000,0.0000,0.0435,0.0000,23.0000,0.0435
...,...,...,...,...,...,...,...,...,...
2603,989,0.0000,0.7333,0.0000,0.0667,0.2000,0.0000,15.0000,0.2000
2604,99,0.0000,0.4783,0.0000,0.3696,0.1304,0.0217,46.0000,0.1522
2605,991,0.0000,0.6596,0.0000,0.1277,0.1702,0.0426,47.0000,0.2128
2606,994,0.0000,0.3077,0.0000,0.4615,0.1923,0.0385,26.0000,0.2308


In [576]:
# Como vamos comparar o número de professores negros observado e esperado, vamos renomear a coluna "negra" para não
# ficar confuso
dados2019_difporc.rename({"negra": "negra_observado"}, axis=1, inplace=True)

In [577]:
# juntando com a tabela ies_e_uf para ter o código da UF, a sigla da IES e o nome da IES
dados2019_difporc = pd.merge(dados2019_difporc, ies_e_uf, on=["CO_IES"], how="left")

In [579]:
# juntando com a tabela uf_e_codigo para ter o nome da UF
dados2019_difporc = pd.merge(dados2019_difporc, uf_e_codigo, on=["CO_UF"], how="left")

In [580]:
# juntando com a tabela esperado_raca_uf para ter a porcentagem de habitantes negros em cada uf
dados2019_difporc = pd.merge(dados2019_difporc, esperado_raca_uf, on=["nome_uf"], how="left")

In [582]:
# a 'difereça' vai ser a divisão entre o valor observado e o esperado. Se essa 'diferença' foi => 1, significa que
# a universidade tem proporção de professores negros igual ou superior que a de habitantes negros na UF em que está 
# instalada. Se for <1, significa que a IES tem menos professores negros que o ideal

dados2019_difporc["diferenca"] = dados2019_difporc["negra_observado"] / dados2019_difporc["negra_esperado"]

In [584]:
# Quero incluir a informação sobre categoria administrativa na tabela. Primeiro vou selecionar só a categoria administrativa
# e o Código da Ies para depois juntar essas duas tabelas
cat_admin = dadosies2019.loc[:, ["CO_IES", "NOME_CATEGORIA_ADMINISTRATIVA"]]

In [585]:
# juntando as tabelas
dados2019_difporc = pd.merge(dados2019_difporc, cat_admin, on="CO_IES", how="left")

In [586]:
dados2019_difporc

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra_observado,CO_UF,SG_IES,NO_IES,nome_uf,negra_esperado,diferenca,NOME_CATEGORIA_ADMINISTRATIVA
0,1,0.0627,0.5847,0.0025,0.1042,0.2441,0.0017,"2,360.0000",0.2458,51,UFMT,UNIVERSIDADE FEDERAL DE MATO GROSSO,Mato Grosso,0.7000,0.3511,federal
1,10,0.0220,0.5715,0.0007,0.3640,0.0319,0.0099,"1,412.0000",0.0418,41,PUCPR,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,Paraná,0.3560,0.1174,privada sem fins lucrativos
2,10016,0.0000,0.5000,0.0000,0.0000,0.5000,0.0000,6.0000,0.5000,23,FACO,FACULDADE OBOÉ - FACO,Ceará,0.7200,0.6944,privada com fins lucrativos
3,10058,0.0000,0.1667,0.0000,0.3571,0.3810,0.0952,42.0000,0.4762,29,NaN,FACULDADE DE TECNOLOGIA DE VALENÇA,Bahia,0.8010,0.5945,privada com fins lucrativos
4,10071,0.0000,0.9565,0.0000,0.0000,0.0435,0.0000,23.0000,0.0435,43,ITEPAFACULDADES,FACULDADE DE TEOLOGIA E CIÊNCIAS HUMANAS,Rio Grande do Sul,0.2060,0.2111,privada sem fins lucrativos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2603,989,0.0000,0.7333,0.0000,0.0667,0.2000,0.0000,15.0000,0.2000,31,ESMA,ESCOLA SUPERIOR EM MEIO AMBIENTE,Minas Gerais,0.6090,0.3284,privada sem fins lucrativos
2604,99,0.0000,0.4783,0.0000,0.3696,0.1304,0.0217,46.0000,0.1522,31,FDCL,FACULDADE DE DIREITO DE CONSELHEIRO LAFAIETE,Minas Gerais,0.6090,0.2499,privada sem fins lucrativos
2605,991,0.0000,0.6596,0.0000,0.1277,0.1702,0.0426,47.0000,0.2128,33,SENAI-CETIQT,FACULDADE SENAI-CETIQT,Rio de Janeiro,0.5410,0.3933,privada sem fins lucrativos
2606,994,0.0000,0.3077,0.0000,0.4615,0.1923,0.0385,26.0000,0.2308,33,FDC,Faculdade de Duque de Caxias,Rio de Janeiro,0.5410,0.4266,privada sem fins lucrativos


In [587]:
# vou chamar de aceitável as IES com autodeclaração superior a 75% (nao consta <= 0.25) e com mais de 50 professores
dados2019_difporc_aceitavel = dados2019_difporc[(dados2019_difporc["nao consta"] <= 0.25) & (dados2019_difporc["TOTAL_PROF"] >= 50)]

In [588]:
dados2019_difporc_aceitavel

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra_observado,CO_UF,SG_IES,NO_IES,nome_uf,negra_esperado,diferenca,NOME_CATEGORIA_ADMINISTRATIVA
0,1,0.0627,0.5847,0.0025,0.1042,0.2441,0.0017,"2,360.0000",0.2458,51,UFMT,UNIVERSIDADE FEDERAL DE MATO GROSSO,Mato Grosso,0.7000,0.3511,federal
5,10116,0.0392,0.4902,0.0000,0.2353,0.2157,0.0196,51.0000,0.2353,51,FATEC SENAI MT,FACULDADE DE TECNOLOGIA SENAI MATO GROSSO,Mato Grosso,0.7000,0.3361,privada sem fins lucrativos
7,1014,0.0000,0.8762,0.0000,0.0190,0.0762,0.0286,105.0000,0.1048,42,BOM JESUS/IELUSC,INSTITUTO SUPERIOR E CENTRO EDUCACIONAL LUTERA...,Santa Catarina,0.1920,0.5456,privada sem fins lucrativos
8,1019,0.0385,0.6154,0.0000,0.0577,0.2885,0.0000,52.0000,0.2885,31,FACTU,FACULDADE DE CIÊNCIAS E TECNOLOGIA DE UNAÍ - F...,Minas Gerais,0.6090,0.4737,privada sem fins lucrativos
9,1021,0.0000,0.7273,0.0000,0.0000,0.2727,0.0000,55.0000,0.2727,26,FACET,FACULDADE DE CIÊNCIAS DE TIMBAÚBA,Pernambuco,0.6760,0.4034,privada sem fins lucrativos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,940,0.0290,0.5362,0.0000,0.1014,0.2899,0.0435,69.0000,0.3333,50,FAMAG,Faculdades Magsul,Mato Grosso do Sul,0.5510,0.6050,privada sem fins lucrativos
2597,95,0.0219,0.4652,0.0139,0.0775,0.3598,0.0616,503.0000,0.4215,23,UVA,UNIVERSIDADE ESTADUAL VALE DO ACARAÚ,Ceará,0.7200,0.5854,estadual
2598,952,0.0000,0.7494,0.0000,0.1822,0.0638,0.0046,439.0000,0.0683,35,UNISANTA,UNIVERSIDADE SANTA CECÍLIA,São Paulo,0.4040,0.1692,privada sem fins lucrativos
2600,967,0.0263,0.9211,0.0000,0.0000,0.0526,0.0000,76.0000,0.0526,35,FATEC/TQ,FACULDADE DE TECNOLOGIA DE TAQUARITINGA,São Paulo,0.4040,0.1303,estadual


In [599]:
dados2019_difporc_aceitavel.sort_values("diferenca", ascending=False)

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra_observado,CO_UF,SG_IES,NO_IES,nome_uf,negra_esperado,diferenca,NOME_CATEGORIA_ADMINISTRATIVA
2070,4096,0.0189,0.6038,0.0000,0.0755,0.2642,0.0377,53.0000,0.3019,43,FTEC Porto Alegre,FACULDADE DE TECNOLOGIA FTEC,Rio Grande do Sul,0.2060,1.4655,privada com fins lucrativos
479,1504,0.0000,0.0581,0.0000,0.0116,0.9070,0.0233,86.0000,0.9302,27,UNINASSAU MACEIÓ,CENTRO UNIVERSITÁRIO MAURÍCIO DE NASSAU DE MACEIÓ,Alagoas,0.7370,1.2622,privada sem fins lucrativos
2120,430,0.0421,0.5086,0.0000,0.0000,0.4379,0.0115,523.0000,0.4493,41,UNIFIL,CENTRO UNIVERSITÁRIO FILADÉLFIA,Paraná,0.3560,1.2622,privada sem fins lucrativos
241,1325,0.0135,0.5000,0.0000,0.0405,0.3784,0.0676,74.0000,0.4459,41,FAP,FACULDADE DE APUCARANA,Paraná,0.3560,1.2527,privada sem fins lucrativos
2166,4530,0.0000,0.0830,0.0000,0.0000,0.9069,0.0101,494.0000,0.9170,27,FITS,CENTRO UNIVERSITÁRIO TIRADENTES,Alagoas,0.7370,1.2442,privada com fins lucrativos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,4810,0.0114,0.8295,0.0000,0.1591,0.0000,0.0000,88.0000,0.0000,43,AMF,FACULDADE ANTÔNIO MENEGHETTI,Rio Grande do Sul,0.2060,0.0000,privada com fins lucrativos
2446,649,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,111.0000,0.0000,41,UNIUV,CENTRO UNIVERSITÁRIO DE UNIÃO DA VITÓRIA,Paraná,0.3560,0.0000,municipal
243,1327,0.0000,0.9231,0.0000,0.0769,0.0000,0.0000,91.0000,0.0000,43,ESPM - POA,ESCOLA SUPERIOR DE PROPAGANDA E MARKETING DE P...,Rio Grande do Sul,0.2060,0.0000,privada sem fins lucrativos
2348,5369,0.0000,0.9423,0.0000,0.0577,0.0000,0.0000,52.0000,0.0000,31,ASMEC,FACULDADES INTEGRADAS ASMEC,Minas Gerais,0.6090,0.0000,privada com fins lucrativos


In [591]:
dados2019_difporc_aceitavel.describe(percentiles=[0.25, 0.50, 0.75, 0.95]).T

,count,mean,std,min,25%,50%,75%,95%,max
amarela,825.0000,0.0111,0.0180,0.0000,0.0000,0.0057,0.0164,0.0392,0.2492
branca,825.0000,0.6852,0.2221,0.0581,0.5333,0.7489,0.8605,0.9561,1.0000
indigena,825.0000,0.0012,0.0042,0.0000,0.0000,0.0000,0.0000,0.0070,0.0672
nao consta,825.0000,0.0826,0.0706,0.0000,0.0187,0.0640,0.1348,0.2187,0.2500
parda,825.0000,0.1965,0.1999,0.0000,0.0392,0.1164,0.3119,0.6049,0.9195
preta,825.0000,0.0234,0.0271,0.0000,0.0056,0.0162,0.0333,0.0741,0.3143
TOTAL_PROF,825.0000,257.5285,457.2590,50.0000,73.0000,117.0000,225.0000,903.8000,"5,924.0000"
negra_observado,825.0000,0.2200,0.2085,0.0000,0.0526,0.1390,0.3457,0.6366,0.9302
negra_esperado,825.0000,0.5240,0.1861,0.1920,0.4040,0.5410,0.6760,0.8080,0.8200
diferenca,825.0000,0.3608,0.2747,0.0000,0.1401,0.2967,0.5267,0.8962,1.4655


In [610]:
# as 'adequadas' são aquelas com diferença maior ou igual a 1
adequadas = dados2019_difporc_aceitavel[dados2019_difporc_aceitavel["diferenca"] >= 1]

# vamos calcular quantos % são adequadas (adequadas / total analisado (aceitáveis))
len(adequadas)/len(dados2019_difporc_aceitavel)*100

2.7878787878787876

In [606]:
# vamos considerar 'margem de erro' aquelas que possuem entre 90%  e 99% do numero adequado de professores
margem_de_erro = dados2019_difporc_aceitavel[(dados2019_difporc_aceitavel["diferenca"] >= 0.90) & (dados2019_difporc_aceitavel["diferenca"] < 1)]
len(margem_de_erro)

18

In [607]:
adequadas

,CO_IES,amarela,branca,indigena,nao consta,parda,preta,TOTAL_PROF,negra_observado,CO_UF,SG_IES,NO_IES,nome_uf,negra_esperado,diferenca,NOME_CATEGORIA_ADMINISTRATIVA
33,1060,0.0088,0.3326,0.0000,0.0088,0.6344,0.0154,454.0000,0.6498,53,IESB,CENTRO UNIVERSITÁRIO DO INSTITUTO DE EDUCAÇÃO ...,Distrito Federal,0.5890,1.1032,privada com fins lucrativos
241,1325,0.0135,0.5000,0.0000,0.0405,0.3784,0.0676,74.0000,0.4459,41,FAP,FACULDADE DE APUCARANA,Paraná,0.3560,1.2527,privada sem fins lucrativos
331,13982,0.0000,0.1690,0.0070,0.0000,0.8169,0.0070,142.0000,0.8239,15,NaN,FACULDADE UNINASSAU BELÉM,Pará,0.8060,1.0223,privada com fins lucrativos
392,14321,0.0000,0.1656,0.0132,0.0132,0.8079,0.0000,151.0000,0.8079,23,UNINASSAU,CENTRO UNIVERSITÁRIO MAURÍCIO DE NASSAU,Ceará,0.7200,1.1221,privada com fins lucrativos
479,1504,0.0000,0.0581,0.0000,0.0116,0.9070,0.0233,86.0000,0.9302,27,UNINASSAU MACEIÓ,CENTRO UNIVERSITÁRIO MAURÍCIO DE NASSAU DE MACEIÓ,Alagoas,0.7370,1.2622,privada sem fins lucrativos
482,1507,0.0000,0.3158,0.0000,0.0000,0.6667,0.0175,57.0000,0.6842,53,FAPRO,FACULDADE PROJEÇÃO DE CEILÂNDIA,Distrito Federal,0.5890,1.1616,privada sem fins lucrativos
668,1661,0.0000,0.3922,0.0000,0.0000,0.5490,0.0588,51.0000,0.6078,53,FAPRO,Faculdade Projeção de Sobradinho,Distrito Federal,0.5890,1.0320,privada sem fins lucrativos
722,1709,0.0000,0.2286,0.0000,0.0429,0.7286,0.0000,140.0000,0.7286,26,UNIT PE,Centro Universitário Tiradentes de Pernambuco,Pernambuco,0.6760,1.0778,privada com fins lucrativos
747,17284,0.0000,0.1231,0.0154,0.0154,0.8154,0.0308,65.0000,0.8462,21,NaN,FACULDADE UNINASSAU SÃO LUÍS,Maranhão,0.8130,1.0408,privada sem fins lucrativos
776,1749,0.0000,0.2113,0.0000,0.0141,0.7042,0.0704,71.0000,0.7746,26,FALUB,FACULDADE LUSO-BRASILEIRA,Pernambuco,0.6760,1.1459,privada sem fins lucrativos


In [608]:
adequadas.value_counts("NOME_CATEGORIA_ADMINISTRATIVA")

NOME_CATEGORIA_ADMINISTRATIVA
privada com fins lucrativos    12
privada sem fins lucrativos    10
estadual                        1
dtype: int64

In [612]:
margem_de_erro.value_counts("NOME_CATEGORIA_ADMINISTRATIVA")

NOME_CATEGORIA_ADMINISTRATIVA
privada com fins lucrativos    12
privada sem fins lucrativos     4
especial                        1
estadual                        1
dtype: int64